In [1]:
import pandapower as pp

In [2]:
import pyomo.environ as pe

In [3]:
import math

In [4]:
net = pp.create_empty_network() 
b1 = pp.create_bus(net, vn_kv=13.2)
b2 = pp.create_bus(net, vn_kv=13.2)
b3 = pp.create_bus(net, vn_kv=13.2)

pp.create_line(net, from_bus=b1, to_bus=b2, length_km=0.8, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b2, to_bus=b3, length_km=1.2, std_type="NAYY 4x50 SE")

pp.create_ext_grid(net, bus=b1)

pp.create_load(net, bus=b3, p_mw=0.350)

0

In [5]:
 pp.runpp(net)

In [6]:
print(net.res_bus.vm_pu)
print(net.res_line)

0    1.000000
1    0.998973
2    0.997425
Name: vm_pu, dtype: float64
   p_from_mw  q_from_mvar   p_to_mw     q_to_mvar     pl_mw   ql_mvar  \
0   0.350909    -0.022814 -0.350545  1.367420e-02  0.000364 -0.009140   
1   0.350545    -0.013674 -0.350000 -9.562987e-10  0.000545 -0.013674   

   i_from_ka   i_to_ka      i_ka  vm_from_pu  va_from_degree  vm_to_pu  \
0   0.015381  0.015360  0.015381    1.000000        0.000000  0.998973   
1   0.015360  0.015348  0.015360    0.998973       -0.010749  0.997425   

   va_to_degree  loading_percent  
0     -0.010749        10.831460  
1     -0.023998        10.816756  


In [7]:
pp.create_sgen(net, b2, p_mw=0.05, q_mvar=0.025, max_p_mw=0.2, max_q_mvar=0.2)

0

In [8]:
pp.runpp(net)

In [9]:
print(net.res_bus.vm_pu)
print(net.res_line)

0    1.000000
1    0.999130
2    0.997582
Name: vm_pu, dtype: float64
   p_from_mw  q_from_mvar   p_to_mw     q_to_mvar     pl_mw   ql_mvar  \
0   0.300817    -0.047832 -0.300544  3.867856e-02  0.000272 -0.009153   
1   0.350544    -0.013679 -0.350000 -7.725773e-10  0.000544 -0.013679   

   i_from_ka   i_to_ka      i_ka  vm_from_pu  va_from_degree  vm_to_pu  \
0   0.013323  0.013265  0.013323     1.00000        0.000000  0.999130   
1   0.015357  0.015346  0.015357     0.99913       -0.013882  0.997582   

   va_to_degree  loading_percent  
0     -0.013882         9.382118  
1     -0.027127        10.815054  


In [10]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (3 elements)
   - load (1 element)
   - sgen (1 element)
   - ext_grid (1 element)
   - line (2 elements)
 and the following results tables:
   - res_bus (3 elements)
   - res_line (2 elements)
   - res_ext_grid (1 element)
   - res_load (1 element)
   - res_sgen (1 element)


Agregamos los parámetros del modelo a los elementos de la red.

In [46]:
def net_add_optfw(net):
    #agregamos parámetros para el costo inicial en forma de un valor constante ic_0 y un valor función lineal de la potencia ic_1
    net.ext_grid['ic_0_mu'] = 0.0
    net.ext_grid['ic_1_mu'] = 0.0

    #idem para costo operativo
    net.ext_grid['oc_0_mu'] = 0.0
    net.ext_grid['oc_1_mu'] = 0.0

    #potencia disponible
    net.ext_grid['ap_mw'] = 0.5
    
    #potencia entregada
    net.ext_grid['op_mw'] = 0.0
    
    #las restricciones pueden ser ninguna, o una lista de restricciones tipo pyomo
    net.ext_grid['constraints'] = None

    #se repite para la carga y los generadores estáticos
    net.load['ic_0_mu'] = 0.0
    net.load['ic_1_mu'] = 0.0
    net.load['oc_0_mu'] = 0.0
    net.load['oc_1_mu'] = 0.0
    net.load['ap_mw'] = 0.5
    net.load['op_mw'] = 0.0
    net.load['constraints'] = None

    net.sgen['ic_0_mu'] = 0.0
    net.sgen['ic_1_mu'] = 0.0
    net.sgen['oc_0_mu'] = 0.0
    net.sgen['oc_1_mu'] = 0.0
    net.sgen['ap_mw'] = 0.5
    net.sgen['op_mw'] = 0.0
    net.sgen['constraints'] = None

In [47]:
net_add_optfw(net)

Vemos como queda por ejemplo el elemento de red externa:

In [48]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,None,0,1.0,0.0,True,0.0,0.0,0.0,0.0,0.5,0.0,None


Defino una función normal de Python para que nos va a dar la componente lineal del costo de la energía

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

In [49]:
def oc_1_ext_grid(model_status={}):
    #modelo sencillo con dos precios, uno entre 0 a 18 y otro de 18 a 24
    res = 0.0
    sx = 1e-6
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 18.0:
            res = 3600.0*sx
        elif 18 <= h and h < 24.0:
            res = 5400.0*sx
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    return res

In [50]:
#test:
#m_s = {'y': 0, 'd': 180, 'h': 14.0, 'dt': 1.0, 'temp': 12.0, 'wv': 10.0, 'eg': 1.0}

#oc_1_ext_grid(m_s)

In [51]:
#20 hs: 5400
#m_s['h'] = 20.5
#oc_1_ext_grid(m_s)

In [52]:
#25 hs: error
#m_s['h'] = 25
#oc_1_ext_grid(m_s)

In [53]:
net.ext_grid['oc_1_mu'][0] = oc_1_ext_grid

In [54]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,None,0,1.0,0.0,True,0.0,0.0,0.0,<function oc_1_ext_grid at 0x000001D147339820>,0.5,0.0,None


In [55]:
def demand(model_status={}):
    #modelo sencillo en forma de escalones
    #devuelvr la fracción de la carga empleada
    res = 0.0
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 6.0:
            res = 0.2
        elif 6.0 <= h and h < 8.0:
            res = 0.4
        elif 8.0 <= h and h < 18.0:
            res = 0.5
        elif 18.0 <= h and h < 22.0:
            res = 1.0
        elif 22.0 <= h and h < 24.0:
            res = 0.3            
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    return res

In [56]:
net.load['op_mw'][0] = demand

In [57]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,None,2,0.35,0.0,0.0,0.0,NaN,1.0,True,wye,0.0,0.0,0.0,0.0,0.5,<function demand at 0x000001D1499CB4C0>,None


In [58]:
def solar_output(model_status={}):
    #modelo sencillo
    #devuelvr la fracción de la potencia entregada en función del tiempo
    #no toma en cuenta la radicación solar ni la temperatura
    res = 0.0
    if 'h' in model_status:
        h = model_status['h']
        if 0.0 <= h and h < 8.0:
            res = 0.0
        elif 8.0 <= h and h < 18.0:
            res = math.exp(-(h-13.0)**2/8)
        elif 18.0 <= h and h < 24.0:
            res = 0.0
        else:
            raise ValueError("Hour outside model range")
    else:
        raise ValueError("Hour not defined")
    
    return res

In [59]:
#test:
#m_s = {'y': 0, 'd': 180, 'h': 43.0, 'dt': 1.0, 'temp': 12.0, 'wv': 10.0, 'eg': 1.0}
#solar_output(m_s)

In [60]:
net.sgen['name'][0] = 'PV1'
net.sgen['ap_mw'][0] = solar_output
net.sgen['ic_0_mu'][0] = 0.02
net.sgen['ic_1_mu'][0] = 3*0.75
net.sgen['oc_0_mu'][0] = 0.0
net.sgen['oc_1_mu'][0] = 500e-6


In [61]:
net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,max_p_mw,max_q_mvar,ic_0_mu,ic_1_mu,oc_0_mu,oc_1_mu,ap_mw,op_mw,constraints
0,PV1,1,0.05,0.025,NaN,1.0,True,wye,True,0.2,0.2,0.02,2.25,0.0,0.0005,<function solar_output at 0x000001D1376645E0>,0.0,None


Primero construyo un modelo manualmente:

Es un modelo de dimensionamiento del único DER que es la planta solar.

In [62]:
m = pe.ConcreteModel()

In [63]:
dias = 365
años = 5

In [64]:
#el rango de horas:
T_i = range(24)

In [65]:
#las variables
m.ap_mw_PV= pe.Var(within = pe.NonNegativeReals)

In [66]:
#despacho de solar disponible
m.p_mw_PV = pe.Var(T_i, within = pe.NonNegativeReals)

In [67]:
#energia comprada a la red
##m.p_mw_Ext = pe.Var(T_i, within = pe.NonNegativeReals, bounds = (0, net.ext_grid['ap_mw'][0]))
#si puedo exportar:
m.p_mw_Ext = pe.Var(T_i, bounds = (-net.ext_grid['ap_mw'][0], net.ext_grid['ap_mw'][0]))

In [68]:
#el rango de condiciones:
m_s = {'y': 0, 'd': 180, 'h': 43.0, 'dt': 1.0, 'temp': 12.0, 'wv': 10.0, 'eg': 1.0}

Escenarios_i = []
for t in T_i:
    Escenarios_i.append({'y': 0, 'd': 180, 'h': 1.0*t, 'dt': 1.0, 'temp': 12.0, 'wv': 10.0, 'eg': 1.0})


In [69]:
#los costos iniciales:
costos_iniciales = net.ext_grid['ic_0_mu'][0] + net.load['ic_0_mu'][0] + net.sgen['ic_0_mu'][0] + \
                   net.ext_grid['ic_1_mu'][0]*0 + net.load['ic_1_mu'][0]*0 + m.ap_mw_PV * net.sgen['ic_1_mu'][0]

In [70]:
print(costos_iniciales)

0.02 + 2.25*ap_mw_PV


In [71]:
costos_variables = sum(m.p_mw_PV[t]*net.sgen['oc_1_mu'][0]*dias*años + m.p_mw_Ext[t]*net.ext_grid['oc_1_mu'][0](Escenarios_i[t])*dias*años for t in T_i)

In [72]:
print(costos_variables)

0.9125*p_mw_PV[0] + 6.57*p_mw_Ext[0] + 0.9125*p_mw_PV[1] + 6.57*p_mw_Ext[1] + 0.9125*p_mw_PV[2] + 6.57*p_mw_Ext[2] + 0.9125*p_mw_PV[3] + 6.57*p_mw_Ext[3] + 0.9125*p_mw_PV[4] + 6.57*p_mw_Ext[4] + 0.9125*p_mw_PV[5] + 6.57*p_mw_Ext[5] + 0.9125*p_mw_PV[6] + 6.57*p_mw_Ext[6] + 0.9125*p_mw_PV[7] + 6.57*p_mw_Ext[7] + 0.9125*p_mw_PV[8] + 6.57*p_mw_Ext[8] + 0.9125*p_mw_PV[9] + 6.57*p_mw_Ext[9] + 0.9125*p_mw_PV[10] + 6.57*p_mw_Ext[10] + 0.9125*p_mw_PV[11] + 6.57*p_mw_Ext[11] + 0.9125*p_mw_PV[12] + 6.57*p_mw_Ext[12] + 0.9125*p_mw_PV[13] + 6.57*p_mw_Ext[13] + 0.9125*p_mw_PV[14] + 6.57*p_mw_Ext[14] + 0.9125*p_mw_PV[15] + 6.57*p_mw_Ext[15] + 0.9125*p_mw_PV[16] + 6.57*p_mw_Ext[16] + 0.9125*p_mw_PV[17] + 6.57*p_mw_Ext[17] + 0.9125*p_mw_PV[18] + 9.854999999999999*p_mw_Ext[18] + 0.9125*p_mw_PV[19] + 9.854999999999999*p_mw_Ext[19] + 0.9125*p_mw_PV[20] + 9.854999999999999*p_mw_Ext[20] + 0.9125*p_mw_PV[21] + 9.854999999999999*p_mw_Ext[21] + 0.9125*p_mw_PV[22] + 9.854999999999999*p_mw_Ext[22] + 0.9125*p_mw_

In [73]:
costos = costos_iniciales + costos_variables

In [74]:
print(costos)

0.02 + 2.25*ap_mw_PV + 0.9125*p_mw_PV[0] + 6.57*p_mw_Ext[0] + 0.9125*p_mw_PV[1] + 6.57*p_mw_Ext[1] + 0.9125*p_mw_PV[2] + 6.57*p_mw_Ext[2] + 0.9125*p_mw_PV[3] + 6.57*p_mw_Ext[3] + 0.9125*p_mw_PV[4] + 6.57*p_mw_Ext[4] + 0.9125*p_mw_PV[5] + 6.57*p_mw_Ext[5] + 0.9125*p_mw_PV[6] + 6.57*p_mw_Ext[6] + 0.9125*p_mw_PV[7] + 6.57*p_mw_Ext[7] + 0.9125*p_mw_PV[8] + 6.57*p_mw_Ext[8] + 0.9125*p_mw_PV[9] + 6.57*p_mw_Ext[9] + 0.9125*p_mw_PV[10] + 6.57*p_mw_Ext[10] + 0.9125*p_mw_PV[11] + 6.57*p_mw_Ext[11] + 0.9125*p_mw_PV[12] + 6.57*p_mw_Ext[12] + 0.9125*p_mw_PV[13] + 6.57*p_mw_Ext[13] + 0.9125*p_mw_PV[14] + 6.57*p_mw_Ext[14] + 0.9125*p_mw_PV[15] + 6.57*p_mw_Ext[15] + 0.9125*p_mw_PV[16] + 6.57*p_mw_Ext[16] + 0.9125*p_mw_PV[17] + 6.57*p_mw_Ext[17] + 0.9125*p_mw_PV[18] + 9.854999999999999*p_mw_Ext[18] + 0.9125*p_mw_PV[19] + 9.854999999999999*p_mw_Ext[19] + 0.9125*p_mw_PV[20] + 9.854999999999999*p_mw_Ext[20] + 0.9125*p_mw_PV[21] + 9.854999999999999*p_mw_Ext[21] + 0.9125*p_mw_PV[22] + 9.854999999999999*p_mw

In [75]:
m.value = pe.Objective(
expr = costos_iniciales + costos_variables,
sense = pe.minimize )

In [76]:
def hourly_power_balance(m, t):
    return m.p_mw_Ext[t] + m.p_mw_PV[t] - net.load['op_mw'][0](Escenarios_i[t])*net.load['p_mw'][0] == 0

In [77]:
m.hourly_power_balance = pe.Constraint(T_i, rule = hourly_power_balance)

In [78]:
#dimensionamiento solar:
m.PV_dim = pe.Constraint(T_i, rule = (lambda m, t : m.p_mw_PV[t] <= m.ap_mw_PV*net.sgen['ap_mw'][0](Escenarios_i[t])))

In [79]:
opt = pe.SolverFactory('glpk')

result_obj= opt.solve(m, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\jmsar\AppData\Local\Temp\tmpevojvhha.glpk.raw --wglp C:\Users\jmsar\AppData\Local\Temp\tmpdre7p17w.glpk.glp
 --cpxlp C:\Users\jmsar\AppData\Local\Temp\tmps5ibydiw.pyomo.lp
Reading problem data from 'C:\Users\jmsar\AppData\Local\Temp\tmps5ibydiw.pyomo.lp'...
49 rows, 50 columns, 83 non-zeros
337 lines were read
Writing problem data to 'C:\Users\jmsar\AppData\Local\Temp\tmpdre7p17w.glpk.glp'...
308 lines were written
GLPK Simplex Optimizer, v4.65
49 rows, 50 columns, 83 non-zeros
Preprocessing...
10 rows, 11 columns, 20 non-zeros
Scaling...
 A: min|aij| =  4.394e-02  max|aij| =  1.000e+00  ratio =  2.276e+01
GM: min|aij| =  9.523e-01  max|aij| =  1.050e+00  ratio =  1.103e+00
EQ: min|aij| =  9.523e-01  max|aij| =  1.000e+00  ratio =  1.050e+00
Constructing initial basis...
Size of triangular part is 10
*     0: obj =   3.198305000e+01 inf =   0.000e+00 (10)
*    17: obj =   6.228785985e+00 inf

In [80]:
m.pprint()

4 Set Declarations
    PV_dim_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    hourly_power_balance_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    p_mw_Ext_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    p_mw_PV_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}

3 Var Declarations
    ap_mw_PV : Size=1, Index=None
      